# Global-scale atmospheric moisture and mass budgets in ERA5 pressure level data

* find the way to compute global-scale mass budget (dry air + moisure) on ERA5
* Apply this budget computation in AI weather prediction model as physcis-based constraints. 

In [1]:
import numpy as np
import xarray as xr

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

Files: ERA5 pressure level data from `1979-01-01T00 - 1979-01-01T12`

* `/glade/derecho/scratch/ksha/CREDIT_data/ERA5_pressure_lev/upper_air_test.zarr`
* `/glade/derecho/scratch/ksha/CREDIT_data/ERA5_pressure_lev/surf_test.zarr`

In [3]:
ds_upper = xr.open_zarr('/glade/derecho/scratch/ksha/CREDIT_data/ERA5_pressure_lev/upper_air_test.zarr')
ds_surf = xr.open_zarr('/glade/derecho/scratch/ksha/CREDIT_data/ERA5_pressure_lev/surf_test.zarr')

In [4]:
GRAVITY = 9.80665
R = 6371000  # m
RHO_WATER = 1000.0 # kg/m^3

In [5]:
x = ds_surf['longitude']
y = ds_surf['latitude']
lon, lat = np.meshgrid(x, y)
level_p = 100*np.array(ds_upper['level'])

In [6]:
level_p # Pa or kg/m/s2

array([   100,    200,    300,    500,    700,   1000,   2000,   3000,
         5000,   7000,  10000,  12500,  15000,  17500,  20000,  22500,
        25000,  30000,  35000,  40000,  45000,  50000,  55000,  60000,
        65000,  70000,  75000,  77500,  80000,  82500,  85000,  87500,
        90000,  92500,  95000,  97500, 100000])

In [7]:
# level_diff = np.diff(level_p)
# level_diff_cumsum = np.concatenate(([0], np.cumsum(level_diff)))

In [8]:
q_v = np.array(ds_upper['specific_humidity']) # kg/kg
q_ci = np.array(ds_upper['specific_cloud_ice_water_content'])
q_cl = np.array(ds_upper['specific_cloud_liquid_water_content'])
# not preprocessed 
# q_r = np.array(ds_upper['specific_rain_water_content'])
# q_s = np.array(ds_upper['specific_snow_water_content'])

q = q_v # + q_ci + q_cl + q_r + q_s

u = np.array(ds_upper['u_component_of_wind']) # m/s
v = np.array(ds_upper['v_component_of_wind'])

In [9]:
# def geometric_mean(data):
#     return np.exp(np.mean(np.log(data)))

# def weighted_mean(data, weights, axis, keepdims=False):
    
#     expanded_weights = np.broadcast_to(weights, data.shape)
#     weighted_sum = np.sum(data * expanded_weights, axis=axis, keepdims=keepdims)
#     weights_sum = np.sum(expanded_weights, axis=axis, keepdims=keepdims)
#     return weighted_sum / weights_sum

def weighted_sum(data, weights, axis, keepdims=False):
    '''
    Compute the weighted sum of a given quantity

    Args:
        data: the quantity to be sum-ed
        weights: weights that can be broadcasted to the shape of data
        axis: dims to compute the sum
        keepdims: keepdims

    Returns:
        weighted sum
    '''
    expanded_weights = np.broadcast_to(weights, data.shape)
    return np.sum(data * expanded_weights, axis=axis, keepdims=keepdims)

def pressure_integral(q, level_p, output_shape):
    '''
    Compute the pressure level integral of a given quantity using np.trapz

    Args:
        q: the quantity with dims of (level, lat, lon) or (time, level, lat, lon)
        level_p: the pressure level of q as [Pa] and with dims of (level,)
        output_shape: either (lat, lon) or (time, lat, lon)

    Returns:
        Pressure level integrals of q
    '''
    # (level, lat, lon) --> (lat, lon)
    if len(output_shape) == 2:
        Q = np.empty(output_shape)
        for ix in range(output_shape[0]):
            for iy in range(output_shape[1]):
                Q[ix, iy] = np.trapz(q[:, ix, iy], level_p)
                
    # (time, level, lat, lon) --> (time, lat, lon)
    elif len(output_shape) == 3:
        Q = np.empty(output_shape)
        for i_time in range(output_shape[0]):
            for ix in range(output_shape[1]):
                for iy in range(output_shape[2]):
                    Q[i_time, ix, iy] = np.trapz(q[i_time, :, ix, iy], level_p)
                    
    else:
        print('wrong output_shape')
        raise
        
    return Q
    
def dx_dy(lat, lon):
    '''
    Compute the grid spacing from 2D lat/lon grids using central difference
    for center grids and forward/backward difference for edge grids

    Args:
        lat, lon: 2D arrays of latitude and longitude.

    Return:
        dy, dx: 2D arrays of grid spacings
    '''
    
    # Convert latitude and longitude from degrees to radians
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    
    # Compute the grid spacing in the latitude direction (dy)
    dy = np.zeros_like(lat)
    dy[1:-1, :] = R * (lat_rad[2:, :] - lat_rad[:-2, :]) / 2.0
    dy[0, :] = R * (lat_rad[1, :] - lat_rad[0, :])
    dy[-1, :] = R * (lat_rad[-1, :] - lat_rad[-2, :])
    
    # Compute the grid spacing in the longitude direction (dx)
    dx = np.zeros_like(lon)
    dx[:, 1:-1] = R * np.cos(lat_rad[:, 1:-1]) * (lon_rad[:, 2:] - lon_rad[:, :-2]) / 2.0
    dx[:, 0] = R * np.cos(lat_rad[:, 0]) * (lon_rad[:, 1] - lon_rad[:, 0])
    dx[:, -1] = R * np.cos(lat_rad[:, -1]) * (lon_rad[:, -1] - lon_rad[:, -2])

    return dy, dx

def compute_grid_area(lat, lon):
    '''
    Compute grid cell areas from 2D lat/lon grids

    Args:
        lat, lon: latitude and longitude with dims of (lat, lon)

    Return:
        grid cell area, dims are (lat, lon)
    '''
    dy, dx = dx_dy(lat, lon)
    area = dy*dx

    return area

area = compute_grid_area(lat, lon)

# w_lat = np.cos(np.deg2rad(lat))
w_lat = area #/ np.sum(area)

## Conservation of total dry air mass

Equation on a single air column (flux form equation with unit of kg/m^2/s):

\begin{equation}
\mathbf{\nabla} \cdot \frac{1}{g} \int_{p_0}^{p_1}{\left[\left(1-q\right)\mathbf{v}\right]}dp + \frac{1}{g}\frac{\partial}{\partial t}\int_{p_0}^{p_1}{\left(1-q\right)}dp = 0
\end{equation}

For global sum, the first term (the divergence of vertically integrated dry air mass flux) is zero. So the second term (the time tendency of vertically integrated dry air mass per area) is also zero. 

From the second term = 0, the global sum of dry air mass $\overline{m_d}$ stays unchanged (kg) :

\begin{equation}
m_d = \sum{\frac{1}{g}\int_{p_0}^{p_1}{\left(1-q\right)}dp}
\end{equation}

\begin{equation}
m_d\left(\mathrm{y_input}\right) - m_d\left(\mathrm{y_pred}\right) = 0
\end{equation}

For any residuals of this conservation, we apply multiplicative correction to specific humidty to close the budget:

\begin{equation}
q^*\left(\mathrm{y_pred}\right) = 1 - \left[1 - q\left(\mathrm{y_pred}\right)\right]*\frac{m_d\left(\mathrm{y_input}\right)}{m_d\left(\mathrm{y_pred}\right)}
\end{equation}

This can be modified to consider the vertical variability of specific humidity $q_w$:
\begin{equation}
r = 1 + q_w * \left(\frac{m_d\left(\mathrm{y_input}\right)}{m_d\left(\mathrm{y_pred}\right)} - 1\right)
\end{equation}

\begin{equation}
q^*\left(\mathrm{y_pred}\right) = 1 - r*\left[1 - q\left(\mathrm{y_pred}\right)\right]
\end{equation}

In [10]:
t0 = 0
t1 = 1
q_correct = np.copy(q) # <-- corrected y_pred on q
output_shape = (2,)+lon.shape

In [11]:
# an example of std(q) that varies on pressure levels
# top of atmos --> surface
q_std = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
                  4, 4, 4, 4, 4, 2, 2])

q_std_norm = q_std / np.mean(q_std) # geometric_mean(q_std) # q_w

In [12]:
correction_cycle_num = 3 # iterative to handle numrical precision

for i in range(correction_cycle_num):
    
    mass_dry_per_area = pressure_integral(1-q_correct, level_p, output_shape) / GRAVITY # kg/m^2
    mass_dry_sum = weighted_sum(mass_dry_per_area, w_lat, axis=(1, 2), keepdims=False) # kg
    
    # ----------------------------------------------------------------------- #
    # check residual term
    mass_dry_res = mass_dry_sum[t0] - mass_dry_sum[t1]
    print('Residual to conserve the dry air mass [kg]: {}'.format(mass_dry_res))
    print('Ratio to the total amount of air [kg/kg]: {}'.format(mass_dry_res/5.148e18))
    
    # ----------------------------------------------------------------------- #
    # correction
    print('Correction iter {}'.format(i))
    # q_correct_ratio = mass_dry_sum[t0] / mass_dry_sum[t1] # no p level weighting
    q_correct_ratio = 1 + q_std_norm * (mass_dry_sum[t0] / mass_dry_sum[t1] - 1)
    q_correct[t1, ...] = 1 - (1 - q_correct[t1, ...]) * q_correct_ratio[:, None, None]

# final checks
mass_dry_per_area = pressure_integral(1-q_correct, level_p, output_shape) / GRAVITY
mass_dry_sum = weighted_sum(mass_dry_per_area, w_lat, axis=(1, 2), keepdims=False)

# ----------------------------------------------------------------------- #
# check residual term
mass_dry_res = mass_dry_sum[t0] - mass_dry_sum[t1]
print('Residual to conserve the dry air mass [kg]: {}'.format(mass_dry_res))
print('Ratio to the total amount of air [kg/kg]: {}'.format(mass_dry_res/5.148e18))

Residual to conserve the dry air mass [kg]: 1640762961920.0
Ratio to the total amount of air [kg/kg]: 3.1871852407148405e-07
Correction iter 0
Residual to conserve the dry air mass [kg]: -241162088448.0
Ratio to the total amount of air [kg/kg]: -4.684578252680653e-08
Correction iter 1
Residual to conserve the dry air mass [kg]: 43565337600.0
Ratio to the total amount of air [kg/kg]: 8.462575291375292e-09
Correction iter 2
Residual to conserve the dry air mass [kg]: 43565337600.0
Ratio to the total amount of air [kg/kg]: 8.462575291375292e-09


In [13]:
q_correct_ratio # r

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.99999999, 0.99999999, 0.99999999, 0.99999999, 0.99999999,
       0.99999999, 0.99999999, 0.99999999, 0.99999999, 0.99999999,
       0.99999999, 0.99999999, 0.99999999, 0.99999999, 0.99999999,
       0.99999999, 0.99999999, 0.99999999, 0.99999999, 0.99999999,
       0.99999999, 0.99999999, 0.99999999, 0.99999999, 0.99999999,
       0.99999999, 0.99999999])

## Moisture budget

Equation on a single air column (flux form equation with unit of kg/m^2/s):

\begin{equation}
\mathbf{\nabla} \cdot \frac{1}{g} \int_{0}^{p_s}{\left(\mathbf{v}q\right)}dp = -\frac{1}{g}\frac{\partial}{\partial t}\int_{0}^{p_s}{q}dp - E - P
\end{equation}

For global sum, the first term (the divergence of integrated moisture flux) is zero. So the second term (the time tendency of total column water $Q$) is balanced by evaporation $E$ and precipitation $P$ (kg/s):

\begin{equation}
\overline{\left(\frac{\partial Q}{\partial t}\right)} =  - \overline{E} - \overline{P}
\end{equation}

For any residuals of this conservation, we use precipitation to close the budge:

\begin{equation}
\overline{P}^* = \overline{\left[\frac{Q\left(\mathrm{y_pred}\right) - Q\left(\mathrm{y_input}\right)}{\mathrm{second}}\right]} - \overline{E}
\end{equation}

\begin{equation}
P^* = P * \frac{\overline{P}^*}{\overline{P}}
\end{equation}

In [14]:
t0 = 0
t1 = 1
N_seconds = 3600 # hourly data
output_shape = (2,)+lon.shape

In [15]:
precip = np.array(ds_surf['total_precipitation'])
evapor = np.array(ds_surf['evaporation'])

precip_flux = precip[t1, ...] * RHO_WATER / N_seconds # m/hour --> kg/m^2/s, positive
evapor_flux = evapor[t1, ...] * RHO_WATER / N_seconds # kg/m^2/s, negative

precip_correct = np.copy(precip_flux) # <-- corrected y_pred on precip

In [16]:
correction_cycle_num = 1

# pre-compute TWC
TWC = pressure_integral(q_correct, level_p, output_shape) / GRAVITY # kg/m^2
dTWC_dt = (TWC[t1, ...] - TWC[t0, ...]) / N_seconds # kg/m^2/s
TWC_sum = weighted_sum(dTWC_dt, w_lat, axis=(0, 1), keepdims=False) # kg/s

# pre-compute evaporation
E_sum = weighted_sum(evapor_flux, w_lat, axis=(0, 1), keepdims=False) # kg/s

for i in range(correction_cycle_num):
    
    P_sum = weighted_sum(precip_correct, w_lat, axis=(0, 1), keepdims=False) # kg/s
    residual = -TWC_sum - E_sum - P_sum # kg/s
    print('Residual to conserve moisture budge [kg/s]: {}'.format(residual))
    
    P_correct = P_sum + residual # kg/s
    P_correct_ratio = (P_sum + residual) / P_sum
    
    precip_correct = precip_correct * P_correct_ratio

# final checks
P_sum = weighted_sum(precip_correct, w_lat, axis=(0, 1), keepdims=False)
residual = -TWC_sum - E_sum - P_sum
print('Residual to conserve moisture budge [kg/s]: {}'.format(residual))

Residual to conserve moisture budge [kg/s]: 882060249.191227
Residual to conserve moisture budge [kg/s]: -38.808773040771484


In [17]:
P_correct_ratio

0.9451103285649111

## Conclosions

* ERA5 conserves global mean quantities well with very little residuals.
* The correction method can close the budge subjet to numerical precision.
* Possible improvement: the corrections were applied equally to all lat/lon gids.
    * (1) 3D corrections can be applied to consider the spatial variability of specific humdity and precipitation flux.
    * (2) Alternatively, such spatial variability can be considered using single-column-based budget calculations.

## Net step

* Clarify needed ERA5 variables --> Prepare ERA5 pressure level data.
* Multi-year testing on ERA5 to check the consistency of its global budget.
* Convert the current numpy functions to pytorch and let it consider batch size dimension.

## Pytorch version (x)

In [ ]:
lon_tensor = torch.from_numpy(lon)
lat_tensor = torch.from_numpy(lat)
level_tensor = torch.from_numpy(level_p)
q_tensor = torch.from_numpy(np.array(q))